# Model Training with Azure ML

An easy-to-use notebook goes step by step through the process of running a large grid search on AzureML. Based on the workflow presented in [this](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=cli) tutorial published by Microsoft.

**The following are required before running this notebook:**

- An Azure account with an active subscription.
- An Azure ML workspace.
- A Compute Cluster. 
- Installed Azure Machine Learning Python SDK v2

A handy guide on how to set up AzureML can be found [here](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources#create-the-workspace).

# 1. Authenticate and connect to AzureML Workspace 

In [64]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential,AzureCliCredential
from azure.ai.ml import MLClient

from azure.ai.ml.automl import SearchSpace, ClassificationPrimaryMetrics
from azure.ai.ml.sweep import (
    Choice,
    Uniform,
    BanditPolicy,
)

from azure.ai.ml import automl

**Note:** You must first authenticate with azure the CLI. Replace the subscription_id, resource_group (related to your AzureML workspace and storage) and workspace in the field below with the appropriate values

In [28]:
credential = AzureCliCredential()

In [29]:
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    subscription_id = "3bef7a06-fecc-47fc-ba25-a7899d62e16c"
    resource_group = "Mini-Tumors-Germany"
    workspace = "Mini-Tumors-Workspace"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)


We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


When setting up your resources pay attention to what region you would like them set in as **this will affect the availability** of different types of compute node. It is best to decide what sort of server/GPU you would like to train on beforehand and check availability.


Information can be found here:


Sizes :https://learn.microsoft.com/en-us/azure/machine-learning/concept-compute-target#supported-vm-series-and-sizes

This is important, Switzerland has very limited options for compute targets:

Regional availability and pricing: https://azure.microsoft.com/en-us/pricing/details/machine-learning/




# 2. Upload Training Data


From the AzureML doccumentation:

*In order to generate models for computer vision tasks with automated machine learning, you need to bring labeled image data as input for model training in the form of an MLTable. You can create an MLTable from labeled training data in JSONL format. If your labeled training data is in a different format (like, pascal VOC or COCO), you can use a conversion script to first convert it to JSONL, and then create an MLTable. Alternatively, you can use Azure Machine Learning's [data labeling tool](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-image-labeling-projects) to manually label images, and export the labeled data to use for training your AutoML model.*


Sample MLTable files that can be used as a template have been provided in the project data direcroty in the training-mltable-folder and validation-mltable-folder respectively.


[Corresponding Microsoft Doccumentation](https://github.com/Azure/azureml-examples/blob/de54247989beee62d1fd89c2b6f1b89f49ef2f47/sdk/python/assets/data/data.ipynb)



**Before we can upload our data to Azure** we must convert it to the PNG format. A helper script to achieve this is included in the project in the file `convert_png.py`. Run it after populating the augmented data folder by runnning `augment_data.py` . If this step ran correctly the ` .../data/converted`  folder should be populated with enumerated PNG images.

In [1]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

my_data = Data(
    path="../data/converted",
    type=AssetTypes.URI_FOLDER,
    description="Images 128x128 Scaled Outliers Removed",
    name="mini-tumors-images-scaled",
)

uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

NameError: name 'ml_client' is not defined

## 3. Reformat Data and Labels

The following two cells should create two .jsonl files (one for training and one for validation) in the apporopriate MLTable folders of the project The train / validation ratio corresponds to 20% of the data going into the validation file.

In [32]:
import numpy as np
import pandas as pd

labels = np.load('../data/converted/images/labels.npy')

new_labs = pd.DataFrame(columns=['filename','labels'])


new_labs['filename'] = [ f'img{i}.png' for i in range(len(labels))]
new_labs['labels'] = labels

new_labs = new_labs.set_index('filename')

new_labs.to_csv('../data/converted/images/labels.csv')
new_labs.to_csv('../data/converted/labels.csv')


### Remember to move to /images subforlder!

In [47]:
import json
import os

# In the big data folder
src_images = "../data/converted"

# We'll copy each JSONL file within its related MLTable folder
# Do this locally in ./data for notebooks
training_mltable_path = "../data/training-mltable-folder/"
validation_mltable_path = "../data/validation-mltable-folder/"

# Reasonable
train_validation_ratio = 5

# Path to the training and validation files
train_annotations_file = os.path.join(training_mltable_path, "train_annotations.jsonl")
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)


# Baseline of json line dictionary
json_line_sample = {
    "image_url": uri_folder_data_asset.path,
    "label": [],
}

print(json_line_sample["image_url"])

labelFile = os.path.join(src_images, "labels.csv")

# Read each annotation and convert it to jsonl line
with open(train_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        with open(labelFile, "r") as labels:
            for i, line in enumerate(labels):
                # Skipping the title line and any empty lines.
                if i == 0 or len(line.strip()) == 0:
                    continue
                line_split = line.strip().split(",")
                if len(line_split) != 2:
                    print("Skipping the invalid line: {}".format(line))
                    continue
                json_line = dict(json_line_sample)
                json_line["image_url"] += f"images/{line_split[0]}"
                json_line["label"] = line_split[1].strip().split(" ")[0]

                # Nothe the train validation split is not random
                if i % train_validation_ratio == 0:
                    # validation annotation
                    validation_f.write(json.dumps(json_line) + "\n")
                else:
                    # train annotation
                    train_f.write(json.dumps(json_line) + "\n")

azureml://subscriptions/3bef7a06-fecc-47fc-ba25-a7899d62e16c/resourcegroups/Mini-Tumors-Germany/workspaces/Mini-Tumors-Workspace/datastores/workspaceblobstore/paths/LocalUpload/73f9826a213885fe962e833cc0ee47f1/converted/


Create MLTable data input using the jsonl files created above. If data is already in the cloud switch the commented and uncommented sections of the following cell.

In [2]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)

'''
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/train")
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/valid")
'''

NameError: name 'training_mltable_path' is not defined

# 3. Compute target setup


From the Azure Docs:

*You will need to provide a [Compute Target](https://docs.microsoft.com/en-us/azure/machine-learning/concept-azure-machine-learning-architecture#computes) that will be used for your AutoML model training. AutoML models for image tasks require [GPU SKUs](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) such as the ones from the NC, NCv2, NCv3, ND, NDv2 and NCasT4 series. We recommend using the NCsv3-series (with v100 GPUs) for faster training. Using a compute target with a multi-GPU VM SKU will leverage the multiple GPUs to speed up training. Additionally, setting up a compute target with multiple nodes will allow for faster model training by leveraging parallelism, when tuning hyperparameters for your model.*

You should edit the size of compute target in the cell below:

In [50]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "gpu-cluster"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="Standard_NC6",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=1,
    )
    begin_create_or_update.begin_create_or_update(compute_config).result()

Found existing compute target.


# 4. Running the Grid Search

Here is an excrept from the Azure Docs explaining non obvious parameters to the function in the following cells:


The `image_classification()` factory function allows user to configure the training job.

- `compute` - The compute on which the AutoML job will run. In this example we are using a compute called 'gpu-cluster' present in the workspace. You can replace it any other compute in the workspace.
- `experiment_name` - The name of the experiment. An experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `primary_metric` - The metric that AutoML will optimize for model selection.
- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data' and 'validation_data'.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g Input(mltable='my_mltable:1') OR Input(mltable=MLTable(local_path="./data"))
The parameter `training_data` must always be provided.


##### set_limits() function parameters:
This is an optional configuration method to configure limits parameters such as timeouts.      
    
- `timeout_minutes` - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. If not specified, the default job's total timeout is 6 days (8,640 minutes).
- `max_trials` - Parameter for maximum number of configurations to sweep. Must be an integer between 1 and 1000. When exploring just the default hyperparameters for a given model algorithm, set this parameter to 1. Default value is 1.
- `max_concurrent_trials` - Maximum number of runs that can run concurrently. If not specified, all runs launch in parallel. If specified, must be an integer between 1 and 100.  Default value is 1.
    NOTE: The number of concurrent runs is gated on the resources available in the specified compute target. Ensure that the compute target has the available resources for the desired concurrency.
    
##### set_training_parameters() function parameters:

- `distributed` - It enable distributed training if compute target contain multiple GPUs. It must be boolean value (default is True).

##### set_sweep() parameters:

- `sampling_algorithm` - Sampling method to use for sweeping over the defined parameter space. Please refer to this [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=SDK-v2#sampling-methods-for-the-sweep) for list of supported sampling methods.
- `early_termination` - Early termination policy to end poorly performing runs. If no termination policy is specified, all configurations are run to completion. Please refer to this [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=SDK-v2#early-termination-policies) for supported early termination policies.

    
More detail on all of the functions below and their hyperparameters are available in the [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters).

In [51]:
# general job parameters
exp_name = "Mini-Tumor-Classification-AML_V1"

In [3]:
image_classification_job = automl.image_classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
    primary_metric=ClassificationPrimaryMetrics.ACCURACY,
)

# NOTE: only accuracy is currently supported as a Primary Metric

NameError: name 'automl' is not defined

In [4]:
# Limits for training. Setting these is a good idea to prevent unexpected/accidental costs.
image_classification_job.set_limits(
    timeout_minutes=1440, #12 hours
    max_trials=20,
    max_concurrent_trials=1,
)

NameError: name 'image_classification_job' is not defined

In [101]:
# Define two search spaces for the transformer model and residual networks. These are the parameters used for the project grid searches.

ss_transformers =   SearchSpace(
              model_name="vitb16r224",
              # According to: https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-hyperparameters
              layers_to_freeze=Choice([0, 5, 9]),
              # 0 for no weighted loss, 1 for weighted loss with sqrt.(class_weights), 2 for weighted loss with class_weights.
              weighted_loss=Choice([0,2]),
              learning_rate=Choice([0.0001, 0.01, 0.01]),
              number_of_epochs=30,
              optimizer="sgd",
        )

ss_cnn = SearchSpace(
          model_name=Choice(["seresnext", "resnet50", "resnet152"]),
          # According to: https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-hyperparameters
          layers_to_freeze=Choice([0, 2]),
          # 0 for no weighted loss, 1 for weighted loss with sqrt.(class_weights), 2 for weighted loss with class_weights.
          weighted_loss=Choice([0,2]),
          learning_rate=Choice([0.0001, 0.001, 0.01]),
          optimizer= Choice(["sgd","adamw"]),
          number_of_epochs=30
        )

image_classification_job.extend_search_space([ss_cnn, ss_transformers]) # Create a search space that is a union of the two above

In [103]:
# Set an early termination policy for individual models, select a sampling algorithm
from azure.ai.ml.sweep import MedianStoppingPolicy

image_classification_job.set_sweep(
    sampling_algorithm="Random",
    early_termination= MedianStoppingPolicy(delay_evaluation = 15, evaluation_interval = 2)
)

In [104]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    image_classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Created job: ImageClassificationJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'target_column_name': 'label', 'validation_data_size': None, 'task_type': <TaskType.IMAGE_CLASSIFICATION: 'ImageClassification'>, 'training_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/8510c5277a97e268ab28ae5e7675d1d3/training-mltable-folder'}, 'validation_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/ef5ffaf8f7ab54efb70d3e073d3586f2/validation-mltable-folder'}, 'test_data': None, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'jovial_fork_74gkzcrl8c', 'description': None, 'tags': {}, 'properties': {'mlflow.source.git.repoURL': 'https://github.com/gakhromov/mini-tumors', 'mlflow.source.git.branch': 'azure_ml', 'mlflow.source.git.commit': '52033550558d310109afedfc4295b10eebf650fa', 'azureml.git.dirty': 'True'}, 'id': '/

In [105]:
# Track the job
ml_client.jobs.stream(returned_job.name)

RunId: jovial_fork_74gkzcrl8c
Web View: https://ml.azure.com/runs/jovial_fork_74gkzcrl8c?wsid=/subscriptions/3bef7a06-fecc-47fc-ba25-a7899d62e16c/resourcegroups/Mini-Tumors-Germany/workspaces/Mini-Tumors-Workspace


Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x143062d60>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14307b820>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14307b6a0>: Failed to establish a new 

ClientAuthenticationError: ERROR: The command failed with an unexpected error. Here is the traceback:
ERROR: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /16bf80d2-3fa8-456d-8ca3-59cab29a2b99/oauth2/v2.0/token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x107270fd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/azure-cli/2.42.0/libexec/lib/python3.10/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/opt/homebrew/Cellar/azure-cli/2.42.0/libexec/lib/python3.10/site-packages/urllib3/util/connection.py", line 73, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/opt/homebrew/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 955, in

Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x14307bac0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Retrying due to transient client side error HTTPSConnectionPool(host='westus2-0.in.applicationinsights.azure.com', port=443): Max retries exceeded with url: /v2.1/track (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x143062a60>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')).
Path /var/folders/1r/s3cngdnd4qn_4zjl5q1dk6200000gq/T/opencensus-python-71b954a8-6b7d-43f5-986c-3d3a6605d803/2022-11-18T221618.576910-d4bf776d.blob.tmp does not exist or is inaccessible.


In [99]:
# View Trials inteface
hd_job = ml_client.jobs.get(returned_job.name + "_HD")
hd_job

Experiment,Name,Type,Status,Details Page
Mini-Tumor-Classification-AML_V1,busy_sun_x51p25t921_HD,sweep,Running,Link to Azure Machine Learning studio


# 5. Track Training Results

The results of training, performance metrics for the models, downloads for the models etc... can be found via the AzureML stuio UI, the link ti witch should be displayed in the cell above. Note that it may be neccesary to explore sub-jobs of the grid search job to find detials on the performance of individual models.